In [ ]:
#!pip install timm

In [ ]:
# Importa bibliotecas necessárias

import PIL
import json
import torch
import pickle
import warnings 
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T

from tqdm import tqdm
from timm import create_model

warnings.filterwarnings('ignore')

# Inicializa modelos e dispositivo para uso

#model_name = "vit_base_patch16_224"
model_name = "vit_base_patch32_224"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device = ", device)
# create a ViT model : https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
model = create_model(model_name, pretrained=True).to(device)

In [ ]:
def load_base(name_arq):
    
    f = open(name_arq, encoding="utf8")
    data = json.load(f)
    
    # Pega apenas as "respondiveis"
    #data = [d for d in data if d["answerable"] == 1]
    
    return data

In [ ]:
def define_transform():
    
    # Define transforms for test
    IMG_SIZE = (224, 224)
    NORMALIZE_MEAN = (0.5, 0.5, 0.5)
    NORMALIZE_STD = (0.5, 0.5, 0.5)
    transforms = [
                  T.Resize(IMG_SIZE),
                  T.ToTensor(),
                  T.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
                  ]

    transforms = T.Compose(transforms)
    
    return transforms

In [ ]:
def get_embedding_vision_transformer(model, img_tensor):
    
    # Divide a imagem em patches
    patches = model.patch_embed(img_tensor)
    
    # Calcula o vetor de positional embedding, para saber a posição correta de cada patch
    pos_embed = model.pos_embed
    
    # Computa o embedding inicial juntamente com suas posições, além de trazer o embedding do token especial "CLS"
    # que representa a imagem inteira e é o primeiro elemento do vetor
    transformer_input = torch.cat((model.cls_token, patches), dim=1) + pos_embed
    
    # Calcula o embedding final, passando pelos blocos de encoder do modelo
    x = transformer_input.clone()
    for i, blk in enumerate(model.blocks):
        x = blk(x)
    x = model.norm(x)
    
    # x é o embedding que irá representar os patches e imagem, sendo na seguinte ordem: imagem, patch 0, patch 1, etc.
    #x = x.reshape(197,768)
    x = x.reshape(50,768)

    return x

In [ ]:
# Visão
"""
{"imagem.jpg": [vetores], 
"imagem2.jpg": [vetores],
...
}
"""

In [ ]:
def get_info_visao(name_arq, model, tam_base):
    
    # Realiza a leitura da base
    data = load_base(name_arq)
    
    #data = data[:tam_base]
    
    # Define diretório onde se encontram as imagens
    dir_img = name_arq.split(".json")[0]
    
    # Define padronização que será feita nas imagens
    transforms = define_transform()
    
    # Irá carregar as informações visuais referentes a cada uma das imagens
    info_visao = {}
    
    for info in tqdm(data):
        
        # Pega o nome da imagem
        name_img = info["image"]
        
        # Faz a leitura da imagem
        img = PIL.Image.open(dir_img+"/"+name_img)
        
        # Padroniza a imagem
        img_tensor = transforms(img).unsqueeze(0).to(device)
        
        # Pega os embeddings referentes a imagem
        embedding_img = get_embedding_vision_transformer(model, img_tensor)
        
        # Atualiza as informações da imagem
        info_visao[name_img] = embedding_img.detach().numpy()
        
    return info_visao

In [ ]:
def save_info_visao(name_arq_in, info_visao):
    
    name_arq_out = name_arq_in.split(".json")[0]+"_info_visao"
    
    file = open(name_arq_out, 'wb')
    pickle.dump(info_visao, file)                   
    file.close()
    
    return 

In [ ]:
%%time

name_arq = "test.json"
info_visao = get_info_visao(name_arq, model, 160)
save_info_visao(name_arq, info_visao)

#### Fontes

https://colab.research.google.com/drive/1jIflL9-gktbXq_2cEE_KM7yYq2PaOKRM?authuser=1#scrollTo=8cRyNhMQaAyh

https://khvmaths.medium.com/vision-transformer-understanding-the-underlying-concept-83d699d71180